## Using the Home Assistant Python API
This notebook shows how you can work the [Home Assistant Python API](https://home-assistant.io/developers/python_api/) inside a Jupyter notebook.

### Setup

In [1]:
import homeassistant.remote as remote

In [2]:
HOST = '127.0.0.1'
PASSWORD = 'YOUR_PASSWORD'

api = remote.API(HOST, PASSWORD)

### Validation of the API is ready to use

In [3]:
print(remote.validate_api(api))

ok


### Retrieve the configuration
The Home Assistant Python API is not complete. Think of it as a sub-set of the [RESTful API](https://home-assistant.io/developers/rest_api/). This means that if a certain option is not available you can use Python's `requests` module to retrieve the data. 

`remote.get_config(api)` is available to get the configuration details about a Home Assistant instance.

In [4]:
print(remote.get_config(api))

{'components': {'sun', 'python_script', 'group', 'light.tradfri', 'config.core', 'binary_sensor', 'discovery', 'automation', 'frontend', 'input_boolean', 'binary_sensor.mqtt', 'sensor.random', 'http', 'config.script', 'config.automation', 'light', 'telegram_bot', 'switch', 'updater', 'tradfri', 'config.group', 'logbook', 'api', 'conversation', 'weather', 'sensor.cpuspeed', 'config', 'history', 'config.customize', 'sensor', 'binary_sensor.template', 'mqtt', 'notify', 'websocket_api', 'switch.mqtt', 'weather.zamg', 'recorder'}, 'config_dir': '/home/fab/.homeassistant', 'elevation': 560, 'latitude': 46.9509, 'location_name': 'Home', 'longitude': 7.4463, 'time_zone': 'Europe/Zurich', 'unit_system': {'length': 'km', 'mass': 'g', 'temperature': '°C', 'volume': 'L'}, 'version': '0.53.0.dev0', 'whitelist_external_dirs': ['/home/fab/.homeassistant/www', '/home/fab/.homeassistant/']}


The same details can be retrieved from the [RESTful API](https://home-assistant.io/developers/rest_api/).

In [5]:
import requests

url = 'http://{}:8123/api/{}'.format(HOST, 'config')
headers = {
    'x-ha-access': PASSWORD,
    'content-type': 'application/json',
}

config = requests.get(url, headers=headers).json()
print(config)

{'components': ['group', 'recorder', 'config.customize', 'sensor.random', 'python_script', 'api', 'updater', 'config.group', 'config', 'http', 'config.automation', 'light', 'notify', 'frontend', 'weather.zamg', 'binary_sensor.template', 'weather', 'sensor.cpuspeed', 'history', 'light.tradfri', 'binary_sensor', 'sensor', 'mqtt', 'telegram_bot', 'binary_sensor.mqtt', 'logbook', 'websocket_api', 'switch.mqtt', 'sun', 'tradfri', 'discovery', 'automation', 'conversation', 'config.script', 'config.core', 'switch', 'input_boolean'], 'config_dir': '/home/fab/.homeassistant', 'elevation': 560, 'latitude': 46.9509, 'location_name': 'Home', 'longitude': 7.4463, 'time_zone': 'Europe/Zurich', 'unit_system': {'length': 'km', 'mass': 'g', 'temperature': '°C', 'volume': 'L'}, 'version': '0.53.0.dev0', 'whitelist_external_dirs': ['/home/fab/.homeassistant/www', '/home/fab/.homeassistant/']}


### Display the current location

In [6]:
# This requires that geopy is installed, eg. pip3 install geopy
from geopy.geocoders import Nominatim

geolocator = Nominatim()
location = geolocator.reverse("{}, {}".format(config.get('latitude'), config.get('longitude')))
location

Location(NMS Bern - Schulanlage Aarhof, Langmauerweg, Gelbes Quartier, Stadtteil I, Bern, Verwaltungskreis Bern-Mittelland, Verwaltungsregion Bern-Mittelland, Bern/Berne, 3011, Schweiz/Suisse/Svizzera/Svizra, (46.95047795, 7.44604083152511, 0.0))

In [7]:
factor = 0.001
box = ((config.get('latitude') - factor, config.get('longitude') - factor),
       (config.get('latitude') + factor, config.get('longitude') + factor))
marker = (config.get('latitude'), config.get('longitude'))

url = '{0}/export/embed.html?bbox={1}%2C{2}%2C{3}%2C{4}&amp;layer=mapnik&amp;marker={5}%2C{6}'.format(
            'https://www.openstreetmap.org', box[0][1], box[0][0], box[1][1], box[1][0], marker[0], marker[1])

In [8]:
iframe = '<iframe width="720" height="280" frameborder="0" scrolling="no" marginheight="0" marginwidth="0" src={} style="border: 1px solid black"></iframe>'.format(url)

In [9]:
from IPython.display import display, HTML
display(HTML(iframe))

There is also a so-called `magics` present for HTML. This will allow us to include a IFRAME as well. 

### Display the state of an entity

In [10]:
cpu = remote.get_state(api, 'sensor.cpu_speed')

In [11]:
print(cpu)

<state sensor.cpu_speed=0.5; Brand=Intel(R) Core(TM) i5-6200U CPU @ 2.30GHz, GHz Advertised=2.3, arch=X86_64, friendly_name=CPU speed, icon=mdi:pulse, unit_of_measurement=GHz @ 2017-08-27T07:24:09.399393+00:00>


In [12]:
print('{} is {} {}.'.format(cpu.attributes['friendly_name'],
                            cpu.state,
                            cpu.attributes['unit_of_measurement']
                            )
      )

CPU speed is 0.5 GHz.
